In [1]:
import numpy as np

import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F 
from torch.utils.data import DataLoader

from dataloader import trainloader
from models.base_hovernet import targets


import os
import argparse
import importlib
from glob import glob
from tqdm import tqdm

from config import normal_Config, uniform_Config
from termcolor import colored
from collections import OrderedDict
from models.base_hovernet.utils import crop_to_shape, dice_loss, mse_loss, msge_loss, xentropy_loss


/usr/local/lib/python3.8/dist-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:
data = normal_Config['shape_info'][normal_Config['run_mode']]
data

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Using device: {device}")
# Set random seed
torch.manual_seed(normal_Config['seed'])
np.random.seed(normal_Config['seed'])
# Load dataset
print("Loading dataset...")

run_mode  = normal_Config['run_mode']
input_dataset = trainloader.FileLoader(
    glob(os.path.join(normal_Config['train_dataset_path'], '*.npy')),     
    mode=normal_Config['run_mode'],
    with_type=normal_Config['with_type'],
    setup_augmentor=True, # 이거 True / False 차이 알아보기
    target_gen=[targets.gen_targets, {}],
    **normal_Config['shape_info'][normal_Config['run_mode']]
)


dataloader = DataLoader(
    input_dataset,
    num_workers=2,
    batch_size=normal_Config['batch_size'],
    shuffle=normal_Config['run_mode'] == "train",
    drop_last=normal_Config['run_mode'] == "train",
    # worker_init_fn=worker_init_fn, # TODO: multiGPU에서 
)

In [ ]:
input_dataset[0].keys()

In [ ]:
# # dict_keys(['img', 'tp_map', 'hv_map', 'np_map'])
# print(len(input_dataset))
# indices = [i for i in range(len(input_dataset))]  # 확인하고 싶은 index 리스트
# unique_values = np.unique(np.concatenate([input_dataset[i]['tp_map'].flatten() for i in indices]))
# print(unique_values)


In [ ]:
# module = importlib.import_module(
#     "models.%s.opt" % normal_Config['model_name']
# )
# model_config = module.get_config(normal_Config['nr_type'], normal_Config['model_mode'])

In [ ]:
from models.base_hovernet.opt import get_config
from models.base_hovernet.net_desc import create_model

temp = get_config(8, 'train')
for i in range(len(temp['phase_list'])):
    print(temp['phase_list'][i]['run_info'])

In [ ]:
nr_type = 8
mode = 'original'
first_phase_model = create_model(
    input_ch=3, nr_types=nr_type, 
    freeze=True, mode=mode
),

In [ ]:
DEVICE = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
loss_func_dict = {
        "bce": xentropy_loss,
        "dice": dice_loss,
        "mse": mse_loss,
        "msge": msge_loss,
    }

In [ ]:
def convert_pytorch_checkpoint(net_state_dict):
    variable_name_list = list(net_state_dict.keys())
    is_in_parallel_mode = all(v.split(".")[0] == "module" for v in variable_name_list)
    if is_in_parallel_mode:
        colored_word = colored("WARNING", color="red", attrs=["bold"])
        print(
            (
                "%s: Detect checkpoint saved in data-parallel mode."
                " Converting saved model to single GPU mode." % colored_word
            ).rjust(80)
        )
        net_state_dict = {
            ".".join(k.split(".")[1:]): v for k, v in net_state_dict.items()
        }
    return net_state_dict

In [ ]:
def load_model(model, pretrained_path, device="cuda"):
    if pretrained_path is not None:
        print("Pretrained Path: ", pretrained_path)
        net_state_dict = torch.load(pretrained_path, map_location=device)["desc"]
        
        # Convert PyTorch checkpoint if necessary
        net_state_dict = convert_pytorch_checkpoint(net_state_dict)
        
        # Load state dict to the model
        load_feedback = model.load_state_dict(net_state_dict, strict=False)
        print("Missing Variables: \n", load_feedback[0])
        print("Detected Unknown Variables: \n", load_feedback[1])
    return model


In [ ]:
import torch
import torch.nn.functional as F
from tqdm import tqdm
from collections import OrderedDict

def validate(model, valid_loader, loss_opts, loss_func_dict, device="cuda"):
    model.eval()  # 모델을 평가 모드로 설정
    total_loss = 0
    num_batches = 0

    with torch.no_grad():  # 검증 중에는 Gradient 계산을 하지 않음
        pbar = tqdm(valid_loader, desc="Validation")
        
        for batch in pbar:
            imgs = batch["img"].permute(0, 3, 1, 2).to(device)
            true_np = batch["np_map"].to(device)
            true_hv = batch["hv_map"].to(device)
            true_tp = batch.get("tp_map", None)
            if true_tp is not None:
                true_tp = true_tp.to(device)

            pred_dict = model(imgs)
            pred_dict = OrderedDict(
                [[k, v.permute(0, 2, 3, 1).contiguous()] for k, v in pred_dict.items()]
            )
            pred_dict["np"] = torch.softmax(pred_dict["np"], dim=-1)
            if "tp" in pred_dict:
                pred_dict["tp"] = torch.softmax(pred_dict["tp"], dim=-1)

            loss = 0
            for branch_name, losses in loss_opts.items():
                for loss_name, weight in losses.items():
                    loss_func = loss_func_dict[loss_name]

                    if branch_name == "np":
                        pred_np = pred_dict["np"][..., 1]
                        true_np = true_np.to(torch.int64)
                        true_np_onehot = F.one_hot(true_np, num_classes=2).type(torch.float32)
                        loss += weight * loss_func(pred_np, true_np_onehot[..., 1])

                    elif branch_name == "hv":
                        pred_hv = pred_dict["hv"]
                        if loss_name == "msge":
                            focus = true_np_onehot[..., 1]
                            loss += weight * loss_func(pred_hv, true_hv, focus)
                        else:
                            loss += weight * loss_func(pred_hv, true_hv)

                    elif branch_name == "tp" and true_tp is not None:
                        pred_tp = pred_dict["tp"].permute(0, 3, 1, 2)
                        if not true_tp.dtype == torch.int64:
                            true_tp = true_tp.to(torch.int64)

                        true_tp_onehot = F.one_hot(true_tp, num_classes=pred_tp.shape[1])
                        true_tp_onehot = true_tp_onehot.permute(0, 3, 1, 2).type(torch.float32)
                        loss += weight * loss_func(pred_tp, true_tp_onehot)
            
            total_loss += loss.item()
            num_batches += 1
            pbar.set_postfix(loss=loss.item())

    avg_loss = total_loss / num_batches
    return avg_loss


In [ ]:
import torch
import torch.nn.functional as F
from torch.nn.utils import clip_grad_norm_

# ✅ 모델 초기화 개선 함수 (He Initialization 사용)
def initialize_weights(model):
    for m in model.modules():
        if isinstance(m, torch.nn.Conv2d) or isinstance(m, torch.nn.Linear):
            torch.nn.init.kaiming_normal_(m.weight, mode='fan_out', nonlinearity='relu')
            if m.bias is not None:
                torch.nn.init.constant_(m.bias, 0)

def train_phase(config, phase_idx, model=None, model_path=None, device="cuda"):
    phase_info = config["phase_list"][phase_idx]
    run_info = phase_info["run_info"]
    net_info = run_info["net"]

    # 모델 초기화 (Phase 1이면 모델 새로 생성, Phase 2면 기존 모델 이어받기)
    if model is None:
        model = net_info["desc"]().to(device)
        model.apply(initialize_weights)  # ✅ 모델 초기화 적용 (He Initialization)
    else:
        model = model.to(device)

    optimizer_class, optimizer_params = net_info["optimizer"]
    optimizer = optimizer_class(model.parameters(), **optimizer_params)
    scheduler = net_info["lr_scheduler"](optimizer)

    loss_opts = net_info["extra_info"]["loss"]
    loss_func_dict = {
        "bce": xentropy_loss,
        "dice": dice_loss,
        "mse": mse_loss,
        "msge": msge_loss,
    }

    pretrained_path = net_info["pretrained"]
    if pretrained_path != -1 and pretrained_path is not None:
        if phase_idx == 0:
            state_dict = torch.load(pretrained_path, map_location=device)["desc"]
            model.load_state_dict(state_dict, strict=False)
        elif phase_idx == 1 and model_path is not None:
            checkpoint_path = f"{model_path}/phase1_model.pth"
            state_dict = torch.load(checkpoint_path, map_location=device)["desc"]
            model.load_state_dict(state_dict, strict=False)

    train_dataset = trainloader.FileLoader(
        glob(os.path.join(normal_Config['train_dataset_path'], '*.npy')),     
        mode=normal_Config['run_mode'],
        with_type=normal_Config['with_type'],
        setup_augmentor=True, # 이거 True / False 차이 알아보기
        target_gen=[targets.gen_targets, {}],
        **normal_Config['shape_info'][normal_Config['run_mode']]
    )
    train_loader = DataLoader(
        train_dataset,
        num_workers=2,
        batch_size=normal_Config['batch_size'],
        shuffle=normal_Config['run_mode'] == "train",
        drop_last=normal_Config['run_mode'] == "train",
        # worker_init_fn=worker_init_fn, # TODO: multiGPU에서 
    )
    
    print(f"Train Dataset Size: {len(train_dataset)}")
    for key, value in train_dataset[0].items():
        print(f"{key}: {value.shape}")
    
    valid_dataset = trainloader.FileLoader(
        glob(os.path.join(normal_Config['valid_dataset_path'], '*.npy')),     
        mode=normal_Config['run_mode'],
        with_type=normal_Config['with_type'],
        setup_augmentor=True, # 이거 True / False 차이 알아보기
        target_gen=[targets.gen_targets, {}],
        **normal_Config['shape_info'][normal_Config['run_mode']]
    )
    valid_loader = DataLoader(
        valid_dataset,
        num_workers=2,
        batch_size=normal_Config['batch_size'],
        shuffle=normal_Config['run_mode'] == "train",
        drop_last=normal_Config['run_mode'] == "train",
        # worker_init_fn=worker_init_fn, # TODO: multiGPU에서 
    )

    nr_epochs = phase_info["nr_epochs"]
    for epoch in range(nr_epochs):
        model.train()
        total_loss = 0

        for batch in train_loader:
            imgs = batch["img"].permute(0, 3, 1, 2).to(device)
            true_np = batch["np_map"].to(device)
            true_hv = batch["hv_map"].to(device)
            true_tp = batch.get("tp_map", None)
            if true_tp is not None:
                true_tp = true_tp.to(device)

            optimizer.zero_grad()
            pred_dict = model(imgs)
            pred_dict = {k: v.permute(0, 2, 3, 1) for k, v in pred_dict.items()}
            for key, value in pred_dict.items():
                print(f"Predicted {key} Shape: {value.shape}")
            
            loss = 0
            for branch_name, losses in loss_opts.items():
                for loss_name, weight in losses.items():
                    loss_func = loss_func_dict[loss_name]

                    if branch_name == "np":
                        # ✅ np Branch: Binary Classification (Softmax 적용 후 그대로 사용)
                        pred_np = torch.softmax(pred_dict["np"], dim=-1)  # Softmax 적용 (dim=-1)
                        print(f"[Branch: np] - After Softmax Shape: {pred_np.shape}")  # Expected: (B, W, H, 2)
                        
                        true_np = true_np.to(torch.int64)
                        true_np_onehot = F.one_hot(true_np, num_classes=2).type(torch.float32)  # Shape: (B, W, H, 2)
                        
                        if loss_name == "bce":
                            np_loss = weight * loss_func(pred_np, true_np_onehot)
                        elif loss_name == "dice":
                            np_loss = weight * loss_func(true_np_onehot, pred_np)
                        
                        loss += np_loss
                        
                        # Debug Output
                        print(f"[Branch: np] - Predicted Shape: {pred_np.shape}")
                        print(f"[Branch: np] - Label Shape: {true_np.shape}")
                        print(f"Branch: np, Loss: {loss_name}, Weight: {weight}, Loss Value: {np_loss.item()}")

                    elif branch_name == "hv":
                        # ✅ hv Branch: Regression Task (Horizontal & Vertical Gradients)
                        pred_hv = pred_dict["hv"]  # Output shape: (B, W, H, 2)
                        true_hv_x = true_hv[..., 0]  # Shape: (B, W, H)
                        true_hv_y = true_hv[..., 1]  # Shape: (B, W, H)
                        pred_hv_x = pred_hv[..., 0]  # Shape: (B, W, H)
                        pred_hv_y = pred_hv[..., 1]  # Shape: (B, W, H)

                        if loss_name == "msge":
                            focus = true_np_onehot[..., 1]  # Positive mask
                            hv_loss = weight * loss_func(true_hv, pred_hv, focus)
                        else:  # mse loss
                            hv_loss_x = weight * loss_func(pred_hv_x, true_hv_x)
                            hv_loss_y = weight * loss_func(pred_hv_y, true_hv_y)
                            hv_loss = (hv_loss_x + hv_loss_y) / 2
                        
                        loss += hv_loss
                        
                        # Debug Output
                        print(f"[Branch: hv] - Predicted Shape: {pred_hv.shape}")
                        print(f"[Branch: hv] - Label Shape: {true_hv.shape}")
                        print(f"Branch: hv, Loss: {loss_name}, Weight: {weight}, Loss Value: {hv_loss.item()}")

                    elif branch_name == "tp":
                        # ✅ tp Branch: Multi-Class Classification (e.g., 8 classes)
                        pred_tp = torch.softmax(pred_dict["tp"], dim=-1)  # Output shape: (B, W, H, 8)
                        true_tp = true_tp.to(torch.int64)
                        true_tp_onehot = F.one_hot(true_tp, num_classes=8).type(torch.float32)  # Shape: (B, W, H, 8)
                        
                        if loss_name == "bce":
                            tp_loss = weight * loss_func(pred_tp, true_tp_onehot)
                        elif loss_name == "dice":
                            tp_loss = weight * loss_func(true_tp_onehot, pred_tp)
                        
                        loss += tp_loss
                        
                        # Debug Output
                        print(f"[Branch: tp] - Predicted Shape: {pred_tp.shape}")
                        print(f"[Branch: tp] - Label Shape: {true_tp.shape}")
                        print(f"Branch: tp, Loss: {loss_name}, Weight: {weight}, Loss Value: {tp_loss.item()}")




            # Backpropagation
            loss.backward()
            
            # ✅ Gradient Clipping 적용
            clip_grad_norm_(model.parameters(), max_norm=1.0)
            
            optimizer.step()
            scheduler.step()
            total_loss += loss.item()

        avg_loss = total_loss / len(train_loader)
        print(f"Epoch [{epoch+1}/{nr_epochs}], Loss: {avg_loss:.4f}")

    return model


In [ ]:
def run_training():
    config = get_config(8, 'original')

    # Phase 1 학습
    model = train_phase(config, phase_idx=0, model_path="./checkpoints")
    
    # Phase 2 학습 (Phase 1 모델을 이어받음)
    model = train_phase(config, phase_idx=1, model=model, model_path="./checkpoints")


In [ ]:
run_training()

In [2]:
import torch
import os
from models.base_hovernet.opt import get_config

# ✅ 모델 경로 설정
checkpoint_dir = "./checkpoints"
phase1_path = os.path.join(checkpoint_dir, "phase1_model.tar")
phase2_path = os.path.join(checkpoint_dir, "phase2_model.tar")

# ✅ 모델 로딩 함수 (모델 구조를 그대로 보기 위해 Strict=True 사용 안함)
def load_model(model_path, model_class, device="cuda"):
    model = model_class().to(device)
    checkpoint = torch.load(model_path, map_location=device)
    model.load_state_dict(checkpoint['model_state_dict'], strict=False)
    model.eval()
    return model

# ✅ Config 로드
config = get_config(8, 'original')
net_info = config["phase_list"][0]["run_info"]["net"]

# ✅ 모델 클래스 정의 (HoverNet 모델 로딩)
model_class = net_info["desc"]

# ✅ Phase 1 모델 로드
phase1_model = load_model(phase1_path, model_class, device="cuda")
print("✅ Phase 1 모델 로드 완료")

# ✅ Phase 2 모델 로드
phase2_model = load_model(phase2_path, model_class, device="cuda")
print("✅ Phase 2 모델 로드 완료")

# ✅ 모델의 Layer와 Weight Shape 확인 함수
def print_model_structure(model, model_name="Model"):
    print(f"\n🔍 {model_name} Structure and Weight Shapes:")
    for name, param in model.named_parameters():
        print(f"Layer: {name} | Shape: {param.shape} | Requires Grad: {param.requires_grad}")
    print("\n" + "="*80)

# ✅ Phase 1 모델 구조 확인
print_model_structure(phase1_model, model_name="Phase 1 Model")

# ✅ Phase 2 모델 구조 확인
print_model_structure(phase2_model, model_name="Phase 2 Model")


✅ Phase 1 모델 로드 완료
✅ Phase 2 모델 로드 완료

🔍 Phase 1 Model Structure and Weight Shapes:
Layer: conv0./.weight | Shape: torch.Size([64, 3, 7, 7]) | Requires Grad: True
Layer: conv0.bn.weight | Shape: torch.Size([64]) | Requires Grad: True
Layer: conv0.bn.bias | Shape: torch.Size([64]) | Requires Grad: True
Layer: d0.units.0.conv1.weight | Shape: torch.Size([64, 64, 1, 1]) | Requires Grad: True
Layer: d0.units.0.conv1/bn.weight | Shape: torch.Size([64]) | Requires Grad: True
Layer: d0.units.0.conv1/bn.bias | Shape: torch.Size([64]) | Requires Grad: True
Layer: d0.units.0.conv2.weight | Shape: torch.Size([64, 64, 3, 3]) | Requires Grad: True
Layer: d0.units.0.conv2/bn.weight | Shape: torch.Size([64]) | Requires Grad: True
Layer: d0.units.0.conv2/bn.bias | Shape: torch.Size([64]) | Requires Grad: True
Layer: d0.units.0.conv3.weight | Shape: torch.Size([256, 64, 1, 1]) | Requires Grad: True
Layer: d0.units.1.preact/bn.weight | Shape: torch.Size([256]) | Requires Grad: True
Layer: d0.units.1.pre

In [8]:
import torch
import os

# ✅ 원본 코드에서 저장된 모델 경로 설정
original_model_path = "/mnt/Hover-Net/Hover-Net_original/logs/05/net_epoch=1.tar"  # 원본 코드 모델 경로를 여기에 넣으세요

# ✅ 모델 로딩 함수 (원본 코드 모델 로딩)
def load_original_model(model_path, model_class, device="cuda"):
    model = model_class().to(device)
    checkpoint = torch.load(model_path, map_location=device)
    model.load_state_dict(checkpoint, strict=False)
    model.eval()
    return model

# ✅ Config 로드 (원본 모델의 config 정보로 수정)
config = get_config(8, 'original')
net_info = config["phase_list"][0]["run_info"]["net"]

# ✅ 모델 클래스 정의 (원본 HoverNet 모델)
model_class = net_info["desc"]

# ✅ 원본 모델 로드
original_model = load_original_model(original_model_path, model_class, device="cuda")
print("✅ 원본 모델 로드 완료")

# ✅ 모델의 Layer와 Weight Shape 확인 함수
def print_model_structure(model, model_name="Model"):
    print(f"\n🔍 {model_name} Structure and Weight Shapes:")
    for name, param in model.named_parameters():
        print(f"Layer: {name} | Shape: {param.shape} | Requires Grad: {param.requires_grad}")
    print("\n" + "="*80)

# ✅ 원본 모델 구조 확인
print_model_structure(original_model, model_name="Original Model")


✅ 원본 모델 로드 완료

🔍 Original Model Structure and Weight Shapes:
Layer: conv0./.weight | Shape: torch.Size([64, 3, 7, 7]) | Requires Grad: True
Layer: conv0.bn.weight | Shape: torch.Size([64]) | Requires Grad: True
Layer: conv0.bn.bias | Shape: torch.Size([64]) | Requires Grad: True
Layer: d0.units.0.conv1.weight | Shape: torch.Size([64, 64, 1, 1]) | Requires Grad: True
Layer: d0.units.0.conv1/bn.weight | Shape: torch.Size([64]) | Requires Grad: True
Layer: d0.units.0.conv1/bn.bias | Shape: torch.Size([64]) | Requires Grad: True
Layer: d0.units.0.conv2.weight | Shape: torch.Size([64, 64, 3, 3]) | Requires Grad: True
Layer: d0.units.0.conv2/bn.weight | Shape: torch.Size([64]) | Requires Grad: True
Layer: d0.units.0.conv2/bn.bias | Shape: torch.Size([64]) | Requires Grad: True
Layer: d0.units.0.conv3.weight | Shape: torch.Size([256, 64, 1, 1]) | Requires Grad: True
Layer: d0.units.1.preact/bn.weight | Shape: torch.Size([256]) | Requires Grad: True
Layer: d0.units.1.preact/bn.bias | Shape: to

In [4]:
import torch

# ✅ 모델 구조와 파라미터 Shape 비교 함수
def compare_models(model1, model2, model1_name="Model 1", model2_name="Model 2"):
    # 모델 1의 Layer 정보 가져오기
    model1_layers = {name: param.shape for name, param in model1.named_parameters()}
    model1_requires_grad = {name: param.requires_grad for name, param in model1.named_parameters()}
    
    # 모델 2의 Layer 정보 가져오기
    model2_layers = {name: param.shape for name, param in model2.named_parameters()}
    model2_requires_grad = {name: param.requires_grad for name, param in model2.named_parameters()}
    
    # 모델 구조 차이 확인
    model1_keys = set(model1_layers.keys())
    model2_keys = set(model2_layers.keys())
    
    only_in_model1 = model1_keys - model2_keys
    only_in_model2 = model2_keys - model1_keys
    common_keys = model1_keys & model2_keys

    print(f"\n🔍 Comparing {model1_name} and {model2_name}\n")
    
    if only_in_model1:
        print(f"⚠️ Layers only in {model1_name}: {only_in_model1}")
    if only_in_model2:
        print(f"⚠️ Layers only in {model2_name}: {only_in_model2}")
    
    if not only_in_model1 and not only_in_model2:
        print("✅ Both models have the same layers.")
    
    # Shape & requires_grad 비교
    mismatched_shapes = []
    requires_grad_diff = []
    
    for key in common_keys:
        if model1_layers[key] != model2_layers[key]:
            mismatched_shapes.append((key, model1_layers[key], model2_layers[key]))
        
        if model1_requires_grad[key] != model2_requires_grad[key]:
            requires_grad_diff.append((key, model1_requires_grad[key], model2_requires_grad[key]))
    
    # Shape mismatch 출력
    if mismatched_shapes:
        print("\n⚠️ Layer Shape Mismatches:")
        for key, shape1, shape2 in mismatched_shapes:
            print(f"  - Layer: {key} | {model1_name} Shape: {shape1} | {model2_name} Shape: {shape2}")
    else:
        print("\n✅ All layers have matching shapes.")
    
    # Requires_grad mismatch 출력
    if requires_grad_diff:
        print("\n⚠️ Requires Grad Differences:")
        for key, req1, req2 in requires_grad_diff:
            print(f"  - Layer: {key} | {model1_name} requires_grad: {req1} | {model2_name} requires_grad: {req2}")
    else:
        print("\n✅ All layers have matching requires_grad settings.")



In [6]:
# ✅ Phase 1 모델 로드 (내가 구현한 모델)
phase1_model_path = "/mnt/Hover-Net/Hover-Net_Custom/checkpoints/phase1_model.tar"
phase1_model = load_model(phase1_model_path, model_class, device="cuda")

# ✅ Phase 2 모델 로드 (원본 코드 모델)
phase2_model_path = "/mnt/Hover-Net/Hover-Net_original/logs/05/net_epoch=1.tar"
phase2_model = load_original_model(phase2_model_path, model_class, device="cuda")

# ✅ 모델 비교 실행
compare_models(phase1_model, phase2_model, model1_name="My Model (Phase 1)", model2_name="Original Model (Phase 2)")



🔍 Comparing My Model (Phase 1) and Original Model (Phase 2)

✅ Both models have the same layers.

✅ All layers have matching shapes.

✅ All layers have matching requires_grad settings.


In [ ]:
import torch

# .tar 파일 로드
checkpoint_path = "/mnt/Hover-Net/Hover-Net_original/logs/05_phase1/net_epoch=1.tar"
checkpoint = torch.load(checkpoint_path, map_location='cpu')

# 저장된 key 확인
print("저장된 Key 목록:")
for key in checkpoint.keys():
    print(key)

# 모델 가중치 확인 (모델 state_dict만 확인하고 싶다면)
if 'desc' in checkpoint:
    print("\n모델 파라미터 목록:")
    for param_tensor in checkpoint['model_state_dict']:
        print(param_tensor, "\t", checkpoint['model_state_dict'][param_tensor].size())

# 옵티마이저 확인
if 'optimizer_state_dict' in checkpoint:
    print("\n옵티마이저 상태 확인 성공")

# 학습률 스케줄러 확인
if 'scheduler_state_dict' in checkpoint:
    print("\n스케줄러 상태 확인 성공")


저장된 Key 목록:
desc
optimizer
lr_scheduler


In [11]:
phase1_model_path = "/mnt/Hover-Net/Hover-Net_Custom/checkpoints/phase1_model.tar"
custom_model = torch.load(phase1_model_path, map_location='cpu')
print("Custom Model Loaded Successfully")
print("저장된 Key 목록:")
for key in custom_model.keys():
    print(key)

Custom Model Loaded Successfully
저장된 Key 목록:
epoch
model_state_dict
optimizer_state_dict
scheduler_state_dict
loss


In [ ]:
import torch

# Original 모델 로드
original_path = "/mnt/Hover-Net/Hover-Net_original/logs/05_phase1/net_epoch=1.tar"
original_checkpoint = torch.load(original_path, map_location='cpu')

# Custom 모델 로드
custom_path = "/mnt/Hover-Net/Hover-Net_Custom/checkpoints/phase1_model.tar"
custom_checkpoint = torch.load(custom_path, map_location='cpu')

# ==========================
# 1. model_state_dict 비교
# ==========================
original_model_dict = original_checkpoint['desc'] if 'desc' in original_checkpoint else original_checkpoint.get('model_state_dict', {})
custom_model_dict = custom_checkpoint.get('model_state_dict', {})

original_keys = set(original_model_dict.keys())
custom_keys = set(custom_model_dict.keys())

missing_in_custom = original_keys - custom_keys
missing_in_original = custom_keys - original_keys
common_keys = original_keys & custom_keys

# Check for missing keys
if missing_in_custom:
    print("🔍 Keys in Original but not in Custom:")
    print(missing_in_custom)

if missing_in_original:
    print("🔍 Keys in Custom but not in Original:")
    print(missing_in_original)

# Compare common keys
for key in common_keys:
    if not torch.equal(original_model_dict[key], custom_model_dict[key]):
        print(f"❌ Weight Mismatch Found: {key}")
    else:
        print(f"✅ Weight Match: {key}")

# ==========================
# 2. optimizer_state_dict 비교
# ==========================
original_optimizer = original_checkpoint.get('optimizer', original_checkpoint.get('optimizer_state_dict', {}))
custom_optimizer = custom_checkpoint.get('optimizer_state_dict', {})

if original_optimizer.keys() != custom_optimizer.keys():
    print("❌ Optimizer keys do not match!")
    print("Original Optimizer Keys:", original_optimizer.keys())
    print("Custom Optimizer Keys:", custom_optimizer.keys())
else:
    print("Original Optimizer Keys:", original_optimizer.keys())
    print("✅ Optimizer keys match!")

# ==========================
# 3. scheduler_state_dict 비교
# ==========================
original_scheduler = original_checkpoint.get('lr_scheduler', original_checkpoint.get('scheduler_state_dict', {}))
custom_scheduler = custom_checkpoint.get('scheduler_state_dict', {})

if original_scheduler.keys() != custom_scheduler.keys():
    print("❌ Scheduler keys do not match!")
    print("Original Scheduler Keys:", original_scheduler.keys())
    print("Custom Scheduler Keys:", custom_scheduler.keys())
else:
    print("Original Scheduler Keys:", original_scheduler.keys())
    print("✅ Scheduler keys match!")


🔍 Keys in Original but not in Custom:
{'module.d3.units.2.conv1/bn.weight', 'module.decoder.tp.u2.dense.units.0.conv1/bn.bias', 'module.decoder.tp.u3.dense.units.0.conv2.weight', 'module.d0.units.0.conv1/bn.num_batches_tracked', 'module.decoder.hv.u3.conva.weight', 'module.decoder.tp.u3.dense.units.3.preact_bna/bn.running_var', 'module.d0.units.0.conv2/bn.num_batches_tracked', 'module.d1.units.3.conv1/bn.bias', 'module.d2.units.1.conv2/bn.running_mean', 'module.decoder.hv.u2.dense.blk_bna.bn.bias', 'module.d2.units.5.preact/bn.bias', 'module.decoder.np.u3.dense.units.7.conv1/bn.running_mean', 'module.decoder.hv.u3.dense.units.5.preact_bna/bn.bias', 'module.decoder.tp.u2.dense.units.2.conv1/bn.running_mean', 'module.decoder.tp.u2.dense.units.2.preact_bna/bn.running_mean', 'module.decoder.tp.u3.dense.units.2.conv1/bn.running_var', 'module.decoder.tp.u3.dense.units.5.conv1/bn.weight', 'module.decoder.hv.u1.conva.weight', 'module.decoder.hv.u3.dense.units.2.conv1.weight', 'module.d1.units.